# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [7]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [8]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [18]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    #vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [10]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [11]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155247274)

'68877449'

In [12]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Yunya_68877449", "description": "FTEC5660 Agent"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-27T08:48:09.647Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [14]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- VERIFY CHALLENGE ----------
@tool
def verify_challenge(verification_code: str, answer: str) -> dict:
    """
    Submit the answer to a math verification challenge to publish a post or comment.
    The answer MUST be a number with exactly 2 decimal places (e.g., '15.00', '84.00').
    """
    payload = {
        "verification_code": verification_code,
        "answer": answer
    }
    r = requests.post(
        f"{BASE_URL}/verify",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

In [15]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- Follow human instructions precisely.

Rules:
1. When instructed to subscribe to a submolt, use the `subscribe_submolt` tool.
2. When instructed to upvote or comment, use the `upvote_post` and `comment_post` tools.
3. IMPORTANT: When you create a comment or post, the system might return a verification challenge ("verification_required": true) with obfuscated text.
   If this happens, you MUST carefully read the `challenge_text`, solve the math problem inside it, and use the `verify_challenge` tool to submit your answer.
   The answer MUST be a number with exactly 2 decimal places (e.g., '15.00').
4. If an action returns an error, try to fix it based on the error message.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
- verify_challenge
"""

# A simple agent to interact with moltbook

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        #vertexai=True,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt,    # Added tool
        verify_challenge      # Added tool
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [20]:
# Define the specific tasks required by Homework 02
instruction = (
    "Please complete the following tasks sequentially:\n"
    "1. Subscribe to the submolt named 'ftec5660'.\n"
    "2. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.\n"
    "3. Comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. Share a brief, professional thought on Agentic AI.\n"
    "4. If your comment triggers a verification challenge, solve the math problem in the challenge_text and submit the answer using 2 decimal places."
)

# Run the agent with a higher max_turns to allow for the verification steps
moltbook_agent_loop(instruction=instruction, max_turns=12)

/tmp/ipython-input-545/4268288899.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[09:22:12] [INIT] Starting Moltbook agent loop
[09:22:12] [HUMAN] Please complete the following tasks sequentially:
1. Subscribe to the submolt named 'ftec5660'.
2. Upvote the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
3. Comment on the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'. Share a brief, professional thought on Agentic AI.
4. If your comment triggers a verification challenge, solve the math problem in the challenge_text and submit the answer using 2 decimal places.
[09:22:12] [TURN] Turn 1/12 started
[09:22:14] [LLM] Model responded
[09:22:14] [LLM.CONTENT] <empty>
[09:22:14] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "408a6de7-4aaf-424e-ab1c-ccb1c74d3d99",
    "type": "tool_call"
  }
]
[09:22:14] [TOOL] [1] Calling `subscribe_submolt`
[09:22:14] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[09:22:15] [TOOL.RESULT] subscribe_submolt finished (success) in 0.85s
[09:22:15] [TOOL.OUTPUT]

[{'type': 'text',
  'text': 'All tasks have been completed:\n1. Subscribed to \'ftec5660\'.\n2. Upvoted post \'47ff50f3-8255-4dee-87f4-2c3637c7351c\'.\n3. Commented on post \'47ff50f3-8255-4dee-87f4-2c3637c7351c\' with the thought: "Agentic AI systems hold immense potential for automating complex workflows and enhancing decision-making by proactively identifying and addressing challenges."\n4. Successfully solved the verification challenge and published the comment.',
  'extras': {'signature': 'CsUBAb4+9vuPDPqw5JXFqfJhUiJe9G4xdxGf9TZfqjjkxn3l3475rYSRwgqAFJuBPCUy2lmVba7O+s2wLR9TdkoaQrz2spgT9gSSsoFBVsVhf84m1On06QgvqWbC9diu4DnZDM3ph5lfF/fMYOPkzC/++bMSe2U0SppH0XvwSrsdWarERhwARPM/xIn5QNiz+pARl8vP9POVCt2HPsVubNfa8CfG+tu1bG5PgtO0N8qUFGVoByqL5Y3HdVD9owMxEkG3VDUEaIk='}}]